In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv
import datetime as dt     
from sqlalchemy import create_engine
import psycopg2

date = dt.date.today()

In [2]:
# Files to Load
theme_data_to_load = "lego-database/themes.csv"
sets_data_to_load = "lego-database/sets.csv"
# Read Sets and Theme Data Files and store into Pandas Data Frames
themes_data = pd.read_csv(theme_data_to_load)
sets_data = pd.read_csv(sets_data_to_load)

#colors_data
sets_data.head()

set_num                        name  year  theme_id  num_parts
0    00-1             Weetabix Castle  1970       414        471
1  0011-2           Town Mini-Figures  1978        84         12
2  0011-3  Castle 2 for 1 Bonus Offer  1987       199          2
3  0012-1          Space Mini-Figures  1979       143         12
4  0013-1          Space Mini-Figures  1979       143         12

In [3]:
themes_data.head()

id                  name  parent_id
0   18             Star Wars        1.0
1  118           Speed Racer      112.0
2  135          Life On Mars      126.0
3  160  Star Wars Clone Wars      159.0
4  161   Star Wars Episode 2      159.0

In [4]:
comb = pd.DataFrame(sets_data['theme_id'].isin(themes_data['id']))
comb.head()

theme_id
0     False
1     False
2     False
3     False
4     False

In [5]:
comb = sets_data[sets_data['theme_id'].isin(themes_data['id'])]
comb.rename(columns={'name': 'set name'}, inplace=True)
comb = comb.merge(themes_data.set_index('id')['name'], 'left', left_on='theme_id', right_index=True)
comb.groupby('name').count()
comb

C:\Users\wdela\Anaconda3\envs\PythonWebMongo\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


set_num                                 set name  year  theme_id  \
45     10019-1              Rebel Blockade Runner - UCS  2001       174   
54     10026-1                  Naboo Starfighter - UCS  2002       172   
57     10030-1            Imperial Star Destroyer - UCS  2002       174   
95     10075-1                   Spider-Man Action Pack  2002       488   
116    10123-1                               Cloud City  2003       169   
...        ...                                      ...   ...       ...   
11651    VP-12       Star Wars Co-Pack of 7121 and 7151  2000       166   
11656     VP-2       Star Wars Co-Pack of 7110 and 7144  2001       169   
11657     VP-3       Star Wars Co-Pack of 7131 and 7151  2000       166   
11658     VP-4  Star Wars Co-Pack of 7101 7111 and 7171  2000       166   
11662     VP-8       Star Wars Co-Pack of 7130 and 7150  2000       169   

       num_parts                     name  
45          1747  Star Wars Episode 4/5/6  
54           187      Star Wars Episode 1  
57          3115  Star Wars Episode 4/5/6  
95            25               Spider-Man  
116          707  Star Wars Episode 4/5/6  
...          ...                      ...  
11651          2      Star Wars Episode 1  
11656          2  Star Wars Episode 4/5/6  
11657          2      Star Wars Episode 1  
11658          3      Star Wars Episode 1  
11662          2  Star Wars Episode 4/5/6  

[1049 rows x 6 columns]

In [6]:
#testing for warning above using the first line of the merged data set
test_df = themes_data.loc[themes_data["id"] == 174, :]
test_df
test2_df = sets_data.loc[sets_data["set_num"] == '10019-1', :]
test2_df

set_num                         name  year  theme_id  num_parts
45  10019-1  Rebel Blockade Runner - UCS  2001       174       1747

In [7]:
#create and engine to start putting this in SQL database
#Note: will not work unless PostGres has a database added with the name legomovies_db
rds_connection_string = "postgres:postgres@localhost:5432/legomovies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
engine.table_names()

['lego_sets', 'lego_themes', 'movie_name']

In [9]:
comb.to_sql(name='lego_sets', con=engine, if_exists='append', index=False)
comb.to_sql(name='lego_themes', con=engine, if_exists='append', index=False)

In [10]:
 pd.read_sql_query('select * from lego_sets', con=engine).head()

set_num                       set name  year  theme_id  num_parts  \
0  10019-1    Rebel Blockade Runner - UCS  2001       174       1747   
1  10026-1        Naboo Starfighter - UCS  2002       172        187   
2  10030-1  Imperial Star Destroyer - UCS  2002       174       3115   
3  10075-1         Spider-Man Action Pack  2002       488         25   
4  10123-1                     Cloud City  2003       169        707   

                      name  
0  Star Wars Episode 4/5/6  
1      Star Wars Episode 1  
2  Star Wars Episode 4/5/6  
3               Spider-Man  
4  Star Wars Episode 4/5/6

In [11]:
#show that the table was created
engine.table_names()

['lego_sets', 'lego_themes', 'movie_name']